In [2]:
%matplotlib inline

import numpy as np

from scipy.linalg import solve_toeplitz
from numpy.linalg import solve
from jax.numpy.linalg import solve as jsolve

Let's start with some definitions. We have a time series dataset consisting of $N$ samples:

$$
\mathbf{x} = \{x_1, x_2, \dots, x_N\}.
$$

In the case where we need to perform interpolation, the samples $\{x_1, \dots, x_m\}$ and $\{x_{m+k+1}, \dots, x_N\}$ are known, while there are $k$ missing samples $\{x_{m+1}, \dots, x_{m+k}\}$. We can call the known samples:

$$
\mathbf{y} = \{x_1, \dots, x_m, x_{m+k+1}, x_N\},
$$

and the missing samples:

$$
\mathbf{z} = \{x_{m+1}, \dots, x_{m+k}\}.
$$

The samples are assumed to be produced by some [autoregressive process](https://en.wikipedia.org/wiki/Autoregressive_model) given by:

$$
x_{i} = \left(\sum_{j=1}^{p} \theta_j x_{i-j}\right) + e_i, 
$$

where $p$ is the order of the autoregressive mode, $\theta$ are the parameters that define the model and $e_i$ is some additive (stationary, white, Gaussian) noise.

We can put the above equation into vector and matrix form and rearrange to give:

$$
\mathbf{e} = \mathbf{x} - \mathbf{L}\mathbf{\theta},
$$

where (removing the $p$ initial points from the vector $\mathbf{x}$, which are required to produce the $(p+1)^{\rm th}$ point) it could be written in full as:

$$
\left[\begin{array}{c}e_{p+1} \\ e_{p+2} \\ e_{p+3} \\ \vdots \\ e_{N-2} \\ e_{N-1} \\ e_{N}\end{array}\right] = \left[\begin{array}{c}x_{p+1} \\ x_{p+2} \\ x_{p+3} \\ \vdots \\ x_{N-2} \\ x_{N-1} \\ x_{N}\end{array}\right] - 
\left[\begin{array}{cccc}
x_{p} & x_{p-1} & \ldots & x_1 \\
x_{p+1} & x_{p} & \ldots & x_2 \\
x_{p+2} & x_{p+1} & \ldots & x_3 \\
\vdots & \vdots & \vdots & \vdots \\
x_{N-3} & x_{N-4} & \ldots & x_{N-2-p} \\
x_{N-2} & x_{N-3} & \ldots & x_{N-1-p} \\
x_{N-1} & x_{N-2} & \ldots & x_{N-p}
\end{array}\right]
\left[\begin{array}{c}\theta_1 \\ \vdots \\ \theta_p\end{array}\right]
$$

Or, alternatively into the form:

$$
\mathbf{e} = \mathbf{K}\mathbf{x},
$$

where in this case we assume that the unobserved data prior to $x_1$ is zero and can write:

$$
\left[
\begin{array}{c}
e_1 \\ e_2 \\ e_3 \\ e_4 \\ \vdots \\ e_{N-3} \\ e_{N-2} \\ e_{N-1} \\ e_N \end{array}
\right] = 
\left[
\begin{array}{ccccccccc}
1 & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & 0 \\
-\theta_1 & 1 & 0 & 0 & \ldots & 0 & 0 & 0 & 0 \\
-\theta_2 & -\theta_1 & 1 & 0 & \ldots & 0 & 0 & 0 & 0 \\
-\theta_3 & -\theta_2 & -\theta_1 & 1 & \ldots & 0 & 0 & 0 & 0 \\
& & & & \ddots & & & & \\
0 & 0 & 0 & 0 & \ldots & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \ldots & -\theta_1 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & \ldots & -\theta_2 & -\theta_1 & 1 & 0 \\
0 & 0 & 0 & 0 & \ldots & -\theta_3 &-\theta_2 & -\theta_1 & 1 \\
\end{array}
\right]
\left[
\begin{array}{c}
x_1 \\ x_2 \\ x_3 \\ x_4 \\ \vdots \\ x_{N-3} \\ x_{N-2} \\ x_{N-1} \\ x_N \end{array}
\right].
$$

Here $\mathbf{K}$ is a [Toeplitz matrix](https://en.wikipedia.org/wiki/Toeplitz_matrix), i.e., all the diagonal's from left to right have the same value.

For the Gibbs sampler, we must first draw a set of missing samples $\mathbf{z}'$. The conditional distribution from which we can draw these is a multivariate Gaussian with an inverse covariance matrix given by:

$$
\mathbf{C}^{-1} = \frac{\mathbf{D}}{\sigma^2},
$$

where $\sigma$ is an estimate of the standard deviation of the noise in the autoregressive model and $\mathbf{D}$ is a submatrix of $\mathbf{K}^T \mathbf{K}$ corresponding to the missing data vector $z$, i.e., it will be the matrix within $\mathbf{K}^T \mathbf{K}$ from rows $m+1 \rightarrow m+k$ and columns $m+1 \rightarrow m+k$. The mean of the multivariate Gaussian is given by:

$$
\mu = -\mathbf{D}^{-1}\mathbf{B}^T\mathbf{y},
$$

where $\mathbf{B}$ is composed of the submatrices of $\mathbf{K}^T \mathbf{K}$ corresponding to the columns $m+1 \rightarrow m+k$ and the rows from $1 \rightarrow m$ and $m + k + 1 \rightarrow N$.

In [73]:
from scipy.linalg import toeplitz

def Kmatrix(theta, N):
    """
    Function to create the NxN Toeplitz array K.
    
    Parameters
    ----------
    theta: array_like
        The set of autorgression parameters.
    N: int
        The length of the data set.
    
    Returns
    -------
    K: ndarray
        The NxN Toeplitz array
    """
    
    # create top row
    r = np.zeros(N)
    r[0] = 1.0
    
    # create first column
    c = np.zeros(N)
    c[0] = 1.0
    c[1:len(theta) + 1] = theta
    
    return toeplitz(c, r=r)


def Dmatrix(ktk, m, k):
    """
    Get the submatrix of ktk corresponding to the rows and columns
    from m to m+k.
    """
    
    return ktk[m:m + k, m:m + k]


def Bmatrix(ktk, m, k):
    """
    Get the submatrix of ktk corresponing to the columns from m to
    m+k and the rows 1 to m and m+k to N where N is the height of
    the array.
    """
    
    return np.hstack((ktk[m:m + k, :m], ktk[m:m + k, m+k:])).T


def draw_missing_data(theta, sigma, y, m, k, rng=None):
    """
    Draw the missing data from the conditional distribution based
    on estimates of the autoregressive parameters and the noise.
    
    Parameters
    ----------
    theta: array_like
        The set of p autoregressive parameters
    sigma: float
        The noise standard deviation
    y: array_like
        The observed data.
    m: int
        The index giving the start of the missing samples (starting
        from 1).
    k: int
        The number of missing samples
    rng: default_rng
        The Numpy default random number generator
        
    Returns
    -------
    z: array_like
        An set of k missing samples.
    """
    
    # total length of data
    N = len(y) + k
    
    # create K matrix
    K = Kmatrix(theta, N)
    
    # calculate K^T K
    KTK = np.dot(K.T, K)
    
    # get the B and D submatrices of KTK
    B = Bmatrix(KTK, m, k)
    D = Dmatrix(KTK, m, k)
    
    # get the covariance matrix of the multivariate Gaussian
    cov = np.linalg.inv(D / sigma ** 2)
    
    # get the mean of the multivariate Gaussian
    mu = -np.dot(np.linalg.inv(D), np.dot(B.T, y))
    
    if rng is None:
        rng = np.random.default_rng()
    
    return rng.multivariate_normal(mu, cov, method="cholesky")

In [74]:
y = np.random.randn(20)
theta = [-0.3, 0.2, 0.5]
k = 5
m = 7
sigma = 1.0

z = draw_missing_data(theta, sigma, y, m, k)
z

array([-0.69088435,  1.06376234, -0.11178913,  0.3638735 , -0.67003659])

In [60]:
m = 2  # number of points before missing data
k = 3  # number of missing data points

B = Bmatrix(KTK, m, k)
D = Dmatrix(KTK, m, k)

In [62]:
np.linalg.inv(D)

array([[ 0.6673932 , -0.00563904, -0.20217201],
       [-0.00563904,  0.60619731, -0.00563904],
       [-0.20217201, -0.00563904,  0.6673932 ]])

In [61]:
B

array([[ 0.5 , -0.5 ,  0.  ],
       [ 0.02,  0.5 , -0.5 ],
       [-0.5 ,  0.5 ,  0.02],
       [ 0.  , -0.5 ,  0.5 ],
       [ 0.  ,  0.  , -0.5 ],
       [ 0.  ,  0.  ,  0.  ]])

In [20]:
c = y[:, 0]
r = y[0]
cr = (c, r)
%timeit solve_toeplitz(cr, x)

16.3 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%timeit jsolve(y, x)

5.85 µs ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
K = np.array(
    [
        [1, 0, 0, 0, 0, 0, 0, 0],
        [-0.2, 1, 0, 0, 0, 0, 0, 0],
        [0.3, -0.2, 1, 0, 0, 0, 0, 0],
        [0, 0.3, -0.2, 1, 0, 0, 0, 0],
        [0, 0, 0.3, -0.2, 1, 0, 0, 0],
        [0, 0, 0, 0.3, -0.2, 1, 0, 0],
        [0, 0, 0, 0, 0.3, -0.2, 1, 0],
        [0, 0, 0, 0, 0, 0.3, -0.2, 1]
    ]
)

In [24]:
l = np.dot(K.T, K)

In [34]:
c = l[:, 0]
r = l[0]

In [27]:
y = np.random.rand(8)

In [37]:
%timeit solve(l, y)

4.22 µs ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [36]:
%timeit solve_toeplitz(c, y)

20.7 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
